# Finding orthogonal equipartitions for sets in space with $8n$ points

Using floating point precision

## Code

In [1]:
import random
import itertools as it
from tqdm import tqdm
import math

class Vector:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __add__(self, other):
        if isinstance(other, Vector):
            # Add the corresponding components of the two vectors
            result_x = self.x + other.x
            result_y = self.y + other.y
            result_z = self.z + other.z
            return Vector(result_x, result_y, result_z)
        else:
            raise ValueError("Can only add two Vector objects together")

    def __sub__(self, other):
        if isinstance(other, Vector):
            # Subtract the corresponding components of the two vectors
            result_x = self.x - other.x
            result_y = self.y - other.y
            result_z = self.z - other.z
            return Vector(result_x, result_y, result_z)
        else:
            raise ValueError("Can only subtract two Vector objects")

    def __mul__(self, scalar):
        # Multiply the vector by a scalar
        result_x = self.x * scalar
        result_y = self.y * scalar
        result_z = self.z * scalar
        return Vector(result_x, result_y, result_z)

    def cross(self, other):
        # Compute the cross product of two vectors
        result_x = self.y * other.z - self.z * other.y
        result_y = self.z * other.x - self.x * other.z
        result_z = self.x * other.y - self.y * other.x
        return Vector(result_x, result_y, result_z)

    def dot(self, other):
        # Compute the dot product of two vectors
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def norm2(self):
        return self.x * self.x + self.y * self.y + self.z * self.z
    
    def norm(self):
        return math.sqrt(self.norm2())

    def __str__(self):
        return f"({self.x}, {self.y}, {self.z})"

def points_to_vectors(L):
    """
    Converts a list of triples to a list of vectors
    """
    return [Vector(x,y,z) for x,y,z in L]

def generate_random_vectors(num_points=8, min_coord=-10, max_coord=10):
    """
    Generates a list of random 3D vectors
    """
    points = []
    for _ in range(num_points):
        x = random.uniform(min_coord, max_coord)
        y = random.uniform(min_coord, max_coord)
        z = random.uniform(min_coord, max_coord)
        points.append(Vector(x, y, z))
    return points

def split_by_point_normal(point, normal, vectors, tol=1e-9):
    """
    Partitions vectors by the plane normal to normal through point.
    It should be at least sqrt(tol) away from the plane to be counted.
    Returns two lists of indices.
    """
    nn2 = normal.norm2()
    assert nn2>tol, f"Tolerance error norm of {normal} smaller than {tol}"
    Positive = []
    Zero = []
    Negative = []
    for i,v in enumerate(vectors):
        ndv = normal.dot(v) - normal.dot(point)
        if ndv*ndv <= tol*nn2:
            Zero.append(i)
        elif ndv > 0:
            Positive.append(i)
        else:
            Negative.append(i)
    return Negative, Zero, Positive

def make_choices(N,Z,P):
    """
    Takes elments from Z and adds them to N and P so that they have the same size
    """
    assert (len(P)+len(Z)+len(N))%2==0, "Need an even number of total elements."
    if len(N)+len(Z)<len(P) or len(P)+len(Z)<len(N): #Not possible to distribute evenly
        return
    num_negs = (len(P)+len(Z)-len(N))//2 #How many should be added to N
    for new_negs in it.combinations(Z,num_negs):
        newN=N.copy()
        newP=P.copy()
        for i in Z:
            if i in new_negs:
                newN.append(i)
            else:
                newP.append(i)
        yield newN,newP
        
def print_vectors(vectors,as_list=False):
    """
    Prints a list of vectors in a readable form, if as_list==True it prints them as a python list
    """
    if as_list:
        res='['
        for v in vectors:
            res += f'{v}, '
        print(res[:-2]+']')
    else:
        for i,v in enumerate(vectors):
            print(f'{i}: {v}')

def print_partition(partition):
    """
    Prints a partition in a readable form
    """
    for N,P,x,n,Z in partition:
        print(f'Plane defined by {Z},')
        print(f'orthogonal to {n} through {x} separates')
        print(f'{N} and {P}')

def find_321_partition(vectors,tol=1e-9,stop_at_first=True):
    """
    Finds a partition of type 3,2,1
    """
    Equipartitions = []
    for a,b,c in it.combinations(vectors, 3):
        normal1 = (b-a).cross(c-a)
        N1,Z1,P1 = split_by_point_normal(a,normal1,vectors,tol=tol)
        assert len(Z1)==3, f"Points not in general possition: {Z1}"
        for newN1,newP1 in make_choices(N1,Z1,P1):
            #newP1 and newN1 have the same size
            for d,e in it.combinations(vectors, 2):
                normal2 = normal1.cross(e-d)
                N2,Z2,P2 = split_by_point_normal(d,normal2, vectors,tol=tol)
                assert len(Z2)==2, f"Points not in general possition: {Z1}, {Z2}"
                for newN2,newP2 in make_choices(N2,Z2,P2):
                    #newP2 and newN2 have the same size
                    #Check if they split into 4 equal parts
                    cuadrant1 = {v for v in newN1 if v in newN2}
                    if len(cuadrant1)!=len(vectors)//4:
                        continue
                    cuadrant2 = {v for v in newN1 if v not in newN2}
                    cuadrant3 = {v for v in newP1 if v in newN2}
                    normal3 = normal1.cross(normal2)
                    for f in vectors:
                        N3,Z3,P3 = split_by_point_normal(f,normal3, vectors,tol=tol)
                        assert len(Z3)==1, f"Points not in general possition: {Z1}, {Z2}, {Z3}"
                        for newN3,newP3 in make_choices(N3,Z3,P3):
                            #Check if they split into 8 equal parts
                            if len(cuadrant1.intersection(newN3))!=len(vectors)//8 or len(cuadrant2.intersection(newN3))!=len(vectors)//8 or len(cuadrant3.intersection(newN3))!=len(vectors)//8:
                                continue
                            p = ((newN1,newP1,a,normal1,Z1),(newN2,newP2,d,normal2,Z2),(newN3,newP3,f,normal3,Z3))
                            Equipartitions.append(p)
                            if stop_at_first:
                                return Equipartitions
    return Equipartitions

def compute_normals(u1,u2,u3):
    """
    For partitions of type 2,2,2, given vectors u1,u2,u3 searches for orthogonal vectors v1,v2,v3 satisfying that vi is ortogonal to ui
    Returns 0 if none is found
    """
    v11=u1.cross(Vector(1,0,0));v12=u1.cross(Vector(0,1,0));
    v21=u2.cross(Vector(1,0,0));v22=u2.cross(Vector(0,1,0));
    v31=u3.cross(Vector(1,0,0));v32=u3.cross(Vector(0,1,0));
    a1 = v21.dot(v31); b1 = v21.dot(v32); c1 = v22.dot(v31); d1 = v22.dot(v32)
    a2 = v31.dot(v11); b2 = v31.dot(v12); c2 = v32.dot(v11); d2 = v32.dot(v12)
    a3 = v11.dot(v21); b3 = v11.dot(v22); c3 = v12.dot(v21); d3 = v12.dot(v22)
    
    disc = (b1*b2*b3 + c1*c2*c3 - a3*b2*d1 - a2*c3*d1 - a1*b3*d2 + a3*c1*d2 + a2*b1*d3 - a1*c2*d3)**2 - 4*(a2*b1*b3 - a1*b3*c2 + a3*c1*c2 - a2*a3*d1)*(-(b2*c3*d1) + c1*c3*d2 + b1*b2*d3 - a1*d2*d3)
    if disc < 0:
        return 0,0,0
    root = math.sqrt(disc)
    den1 = 2*(a2*b1*b3 - a1*b3*c2 + a3*c1*c2 - a2*a3*d1)
    den2 = 2*(a3*b2*b1 - a2*b1*c3 + a1*c2*c3 - a3*a1*d2)
    den3 = 2*(a1*b3*b2 - a3*b2*c1 + a2*c3*c1 - a1*a2*d3)
    assert abs(den1)>1e-9 and abs(den2)>1e-9 and abs(den3)>1e-9, f"Not in general position {den1,den2,den3}"
    
    x1 = (-(b1*b2*b3) - c1*c2*c3 + a3*b2*d1 + a2*c3*d1 + a1*b3*d2 - a3*c1*d2 - a2*b1*d3 + a1*c2*d3 + root)/den1
    x2 = (-(b1*b2*b3) - c1*c2*c3 - a3*b2*d1 + a2*c3*d1 + a1*b3*d2 + a3*c1*d2 + a2*b1*d3 - a1*c2*d3 + root)/den2
    x3 = (-(b1*b2*b3) - c1*c2*c3 + a3*b2*d1 - a2*c3*d1 - a1*b3*d2 + a3*c1*d2 + a2*b1*d3 + a1*c2*d3 + root)/den3

    return v11*x1 + v12, v21*x2 + v22, v31*x3 + v32

def find_222_partition(vectors,tol=1e-9,stop_at_first=True):
    """
    Finds a partition of type 2,2,2
    """
    Equipartitions = []
    for (a1,b1),(a2,b2),(a3,b3) in it.combinations(it.combinations(vectors, 2),3):
        normal1,normal2,normal3 = compute_normals(b1-a1,b2-a2,b3-a3)
        if normal1 == 0:
            #No partition exists
            continue
        N1,Z1,P1 = split_by_point_normal(a1,normal1,vectors,tol=tol)
        N2,Z2,P2 = split_by_point_normal(a2,normal2,vectors,tol=tol)
        N3,Z3,P3 = split_by_point_normal(a3,normal3,vectors,tol=tol)
        assert len(Z1)==2 and len(Z2)==2 and len(Z3)==2, f"Points not in general possition: {Z1}, {Z2}, {Z3}"
        for newN1,newP1 in make_choices(N1,Z1,P1):
            for newN2,newP2 in make_choices(N2,Z2,P2):
                #Check if they split into 4 equal parts
                cuadrant1 = {v for v in newN1 if v in newN2}
                if len(cuadrant1)!=len(vectors)//4:
                    continue
                cuadrant2 = {v for v in newN1 if v not in newN2}
                cuadrant3 = {v for v in newP1 if v in newN2}
                for newN3,newP3 in make_choices(N3,Z3,P3):
                    #Check if they split into 8 equal parts
                    if len(cuadrant1.intersection(newN3))!=len(vectors)//8 or len(cuadrant2.intersection(newN3))!=len(vectors)//8 or len(cuadrant3.intersection(newN3))!=len(vectors)//8:
                        continue
                    p = ((newN1,newP1,a,normal1,Z1),(newN2,newP2,d,normal2,Z2),(newN3,newP3,f,normal3,Z3))
                    Equipartitions.append(p)
                    if stop_at_first:
                        return Equipartitions
    return Equipartitions

def find_partition(vectors,tol1=1e-9,tol2=1e-9,stop_at_first=True):
    """
    Searches for an orthogonal equipartition 
    """
    Equipartitions = find_321_partition(vectors,tol=tol1,stop_at_first=stop_at_first)
    if len(Equipartitions)>0 and stop_at_first:
        return Equipartitions
    Equipartitions += find_222_partition(vectors,tol=tol2,stop_at_first=stop_at_first)
    return Equipartitions

## Random examples

Looks for orthogonal equipartitions in sets of either $8$ or $16$ random points chosen uniformly and independently in a cube. Saves and counts the examples without equipartitions.

Fails the general position check if it finds a point too close to one of the planes it considers.

### Searching for sets of 8 points with no orthogonal equipartitions.

In [2]:
num_points=8
N=500000

NumExamples = 0
ExamplesNoEquipart8 = []
for TotalTries in tqdm(range(N)):
    vectors = generate_random_vectors(num_points=num_points, min_coord=-100, max_coord=100)
    try:
        partition = find_partition(vectors)
        NumExamples += 1
        if len(partition)==0:
            ExamplesNoEquipart8.append(vectors)
    except Exception as error:
        #If they are not in general position
        continue
print("Sets found in general position:",NumExamples)
print("Sets without equipartition:    ",len(ExamplesNoEquipart8))
print("Proportion:                    ",1.*len(ExamplesNoEquipart8)/NumExamples)
#for vectors in ExamplesNoEquipart8:
#    print_vectors(vectors,as_list=True)

100%|██████████████████████████████████████████████████████████████████████████| 500000/500000 [15:33<00:00, 535.69it/s]

Sets found in general position: 499911
Sets without equipartition:     402
Proportion:                     0.0008041431374784712


In [3]:
print_vectors(ExamplesNoEquipart8[0])

0: (55.46946726181682, 25.57598027846204, -58.73523402768404)
1: (65.48966104149792, 25.418304977811104, -67.7252434235309)
2: (-74.5016451085487, -84.87038437110206, -4.07304515039138)
3: (-68.95394587702081, -87.9571899662107, -12.275451664888479)
4: (38.96637529013654, 93.88248051968779, -0.21116164930243997)
5: (-50.98829002506695, 4.812134923869138, 95.94209805517926)
6: (-71.27019295927417, -72.43981136858471, 78.92049637333852)
7: (73.19607180054956, -17.142008351942778, -53.68290221876588)


### Searching for sets of 16 points with no orthogonal equipartitions.

In [4]:
num_points=16
N=30000

NumExamples = 0
ExamplesNoEquipart16 = []
for TotalTries in tqdm(range(N)):
    vectors = generate_random_vectors(num_points=num_points, min_coord=-100, max_coord=100)
    try:
        partition = find_partition(vectors)
        NumExamples += 1
        if len(partition)==0:
            ExamplesNoEquipart16.append(vectors)
    except Exception as error:
        #If they are not in general position
        continue
print("Sets found in general position:",NumExamples)
print("Sets without equipartition:    ",len(ExamplesNoEquipart16))
print("Proportion:                    ",1.*len(ExamplesNoEquipart16)/NumExamples)
#for vectors in ExamplesNoEquipart16:
#    print_vectors(vectors,as_list=True)

100%|█████████████████████████████████████████████████████████████████████████████| 30000/30000 [20:50<00:00, 23.99it/s]

Sets found in general position: 29892
Sets without equipartition:     5
Proportion:                     0.0001672688344707614


In [5]:
print_vectors(ExamplesNoEquipart16[0])

0: (-7.425134641295799, 36.39972567447961, -28.506215802426425)
1: (83.76951219490357, 85.86840723888287, -65.97373317388352)
2: (47.37791802192646, -61.3561166751174, 45.18265234176454)
3: (-1.8875486535553136, -81.35650883319613, 22.69299766633243)
4: (66.00963628606925, 65.28565377744519, -2.987025402831378)
5: (-27.671354586819035, -84.79860036803598, 82.7592528269125)
6: (-71.15852175313421, 38.876540520526646, -56.29932625995835)
7: (-36.877488276629045, -34.62569608613772, 47.33286339054246)
8: (-13.79899255570838, 85.8072576497562, -17.939373984312198)
9: (17.599024374162298, 59.69333521822358, -29.32901129993506)
10: (40.847284838201915, 97.38315317981579, -92.62258126616858)
11: (-12.853161884639803, -97.97179002531982, 95.72028378795699)
12: (-23.83958868278171, -27.033357811386892, 43.68584233008525)
13: (-81.68733036274497, 44.891532896535324, 43.57908278064605)
14: (54.95841939841074, 54.35930701727577, 17.738422466271146)
15: (-16.052203597721387, 26.635090281625168, 93.